In [ ]:
from CoReDataLoader import  dataset,dataloader, maxlen
numclasses = len(numclasses)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics as metrics

In [ ]:
acc = metrics.Accuracy(task = "multiclass",num_classes = len(numclasses))
auroc = metrics.AUROC(task = "multiclass",num_classes=len(numclasses))
prec = metrics.Precision(task="multiclass",num_classes=len(numclasses))
f1score = metrics.F1Score(task = "multiclass",num_classes=len(numclasses))
avgprec = metrics.AveragePrecision(task = "multiclass",num_classes=len(numclasses))
precrecall =metrics.PrecisionRecallCurve(task = "multiclass")
recall = metrics.Recall(task = "multiclass",num_classes=len(numclasses))
roc = metrics.ROC(task = "multiclass",num_classes=len(numclasses))
conf = metrics.ConfusionMatrix(task = "multiclass",num_classes=len(numclasses))

In [ ]:
class classifier(nn.Module):
    def __init__(self,input_length = maxlen):
        super().__init__()
        self.inlayer = nn.Linear(input_length,4096)
        self.bnorm = nn.BatchNorm1d(4096)
        self.silu = nn.SiLU()
        self.inter1 = nn.Linear(4096,2056)
        self.inter2 = nn.Linear(2056,1024)
        self.inter3 = nn.Linear(1024,1024)
        self.output = nn.Linear(1024,len(numclasses))
        self.softmax = nn.Softmax()

        
    def forward(self,inp):
        inp = inp.to(torch.float32)
        itn = self.inlayer(inp)
        itn = self.bnorm(itn)
        itn = self.silu(itn)
        itn = self.silu(self.inter1(itn))
        itn = self.silu(self.inter2(itn))
        itn = self.silu(self.inter3(itn))
        itn = self.silu(self.output(itn))
        return itn
net = classifier()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr = 1,betas = (0.9,0.999))
epochs = 100
net.to("cpu")

In [ ]:
losses = []
correct = 0
for epoch in range(epochs):
    for batch, (ts,params) in enumerate(dataloader):
        params = params[:,0]
        optimizer.zero_grad()
        outputs = net(ts)
        # print(outputs.shape,params[:,0].to(torch.long).shape)
        loss = criterion(outputs,params.to(torch.long))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        print(f"{epoch+1}/{epochs} {batch+1}/{len(dataloader)} loss = {loss.item()} accuracy = {acc(outputs,params)*100}% {auroc(outputs,params) = } {prec(outputs,params) = } {f1score(outputs,params) = } {avgprec(outputs,params) = } {precrecall(outputs,params) = } {recall(outputs,params) = } {roc(outputs,params) = }\n",end = "\r",flush = True)
        correct = 0
    print(f"Epoch finished: {epoch+1}\n") {}

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()